# Import
该文档通过生成仿真数据来研究DRT数据性值

In [1]:
import os
import re
import sys 
from datetime import datetime
from loguru import logger

import matplotlib.pyplot as plt
%matplotlib qt

import torch
import numpy as np

from scipy.linalg import eig, svd, solve

from cvxopt import matrix, solvers
solvers.options['show_progress'] = False


import pyDRTtools as drt



Initializing pyDRTtools from c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages
['c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\python311.zip', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\DLLs', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN', '', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\win32', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Baihm\\anaconda3\\envs\\EISNN\\Lib\\site-packages\\Pythonwin']
Imported basics
Imported BHT
Imported cli
Imported GUI
Imported HMC
Imported layout
Imported nearest_PD
Imported parameter_selection
Imported peak_analysis
Imported runs
Contents of pyDRTtools package: ['basics', 'GUI', 'layout', 'parameter_selection', 'peak_analysis', 'HMC', 'runs', 'nearest_PD']


# Definition

## Loewner

In [2]:

def Loewner_Framework(f, Z, REALFLAG = True):
    '''==================================================
        Construct Loewner Pencel
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        ==================================================
    '''
    _n = len(f)
    s = 2j * np.pi * f

    # Ensuring the input have an even number of elements 
    # for constructing the model having real entries
    if REALFLAG:
        if _n % 2 != 0:
            _n = _n - 1

    # Left & Right Data for Loewner Framework
    s_left  = s[:_n:2]
    H_left  = Z[:_n:2]
    s_right = s[1:_n:2] 
    H_right = Z[1:_n:2]

    # Construct complex conjugate values for ensuring model having real entries
    if REALFLAG:
        s_left  = np.stack([s_left, s_left.conj()], axis=1).flatten()
        H_left  = np.stack([H_left, H_left.conj()], axis=1).flatten()
        s_right = np.stack([s_right, s_right.conj()], axis=1).flatten()
        H_right = np.stack([H_right, H_right.conj()], axis=1).flatten()

    # Constructing the Loewner Matrix & Shifted Loewner Matrix
    # L   = (H_left[:,None] - H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    # Ls  = (s_left[:,None] * H_left[:,None] - s_right[None,:] * H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    L   = (H_left[None,:] - H_right[:,None]) / (s_left[None,:] - s_right[:,None])
    Ls  = (s_left[None,:] * H_left[None,:] - s_right[:,None] * H_right[:,None]) / (s_left[None,:] - s_right[:,None])

    # Transforming the conplex L & Ls to obtain matrices with real entries
    if REALFLAG:
        _J_diag = np.eye(_n//2)
        _J  = (1/np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])
        _J  = np.kron(_J_diag, _J)

        L       = (_J.conj().T @ L @ _J).real
        Ls      = (_J.conj().T @ Ls @ _J).real
        H_left  = ((_J.T @ H_left).T).real
        H_right = (_J.conj().T @ H_right).real

        
    return L, Ls, H_left, H_right

def state_space_model(L, Ls, H_left, H_right):
    '''==================================================
        Construct state space model from Loewner Pencel
        Parameter: 
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        Returen:
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        ==================================================
    '''
    # rank of the Loewner Pencel
    _rank = np.linalg.matrix_rank(np.concatenate((L, Ls), axis=0))
    Y_L, svd_L, X_L = svd(L, full_matrices=False, lapack_driver='gesvd')
    X_L = X_L.T
    
    # Reduced state space model interpolating the data
    Yk = Y_L[:, :_rank]
    Xk = X_L[:, :_rank]

    Ek = -Yk.T@L@Xk
    Ak = -Yk.T@Ls@Xk
    Bk = Yk.T@H_right
    Ck = H_left.T@Xk

    return Ek, Ak, Bk, Ck

def DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG = True, real_th = 1e5):
    '''==================================================
        Transform state space model to DRT model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        Returen:
            R_i:    R_i from RC pair in DRT
            C_i:    C_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
        ==================================================
    '''
    # Solve Av= λEv & wT A= λ wT E & Res = CvwB/wEv, wEv =  δ
    _pol, _U = eig(Ak, Ek)     # 
    wB = solve(_U, solve(Ek,Bk))
    Cv = Ck @ _U
    _res = Cv * wB

    # Calculate R_i & tau_i
    R_i     = (-_res / _pol)
    C_i     = (1/_res)
    tau_i   = (-1/_pol) 
    # tau_i   = abs(-1/_pol) 

    if REALFLAG:
        real_ratio = np.where(np.abs(tau_i.imag) == 0, np.inf, np.abs(tau_i.real / (tau_i.imag+1e-20)))
        tau_i = np.abs(tau_i[real_ratio > real_th])
        R_i = np.abs(R_i[real_ratio > real_th])
        C_i = np.abs(C_i[real_ratio > real_th])


    return R_i, C_i, tau_i

def DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            R_i:    R_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f
    H = np.array([Ck @ solve(si * Ek - Ak, Bk) for si in s])
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ


def DRT_Reconstruction_DRT(R_i, tau_i, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f  # Broadcasting tau_i to match f
    _RC = R_i[None, :] / (1+s[:,None] * tau_i[None,:])
    H = np.sum(_RC, axis=1)

    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ

def Loe_singularity_analysis(f, Z, REALFLAG = True):
    '''==================================================
        DRT Singularity Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Y, svd_L, X = svd(np.concatenate([L, Ls]), full_matrices=False)

    return svd_L

def Loe_Analysis_Single(f, Z, REALFLAG = True):
    '''==================================================
        DRT Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
            H:          reconstructed impedance values
            res_ReZ:    relative error of real part of impedance values
            res_ImZ:    relative error of imaginary part of impedance values
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Ek, Ak, Bk, Ck = state_space_model(L, Ls, H_left, H_right)
    R_i, C_i, tau_i = DRT_Transform(Ek, Ak, Bk, Ck)
    # H, res_ReZ, res_ImZ = DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z)
    H, res_ReZ, res_ImZ = DRT_Reconstruction_DRT(R_i, tau_i, f, Z)

    return R_i, C_i, tau_i, H, res_ReZ, res_ImZ
    
def Loe_Analysis_Batch(chData, REALFLAG = True):
    '''==================================================
        DRT Analysis for Batch Data
        Parameter: 
            chData:     list of tuples (f, Z) for each channel
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
        ==================================================
    '''
    DRTdata = []
    f = chData[0,0,:]
    for i in range(chData.shape[0]):
        _Z = chData[i,1,:] + 1j*chData[i,2,:]
        R_i, C_i, tau_i, _, _, _ = DRT_Analysis_Single(f, _Z, REALFLAG)
        DRTdata.append((np.array([R_i, C_i, tau_i])))
    return DRTdata



## Tikhonov

In [3]:
def TikDRT(f, Z, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        Tikhonov DRT Deconvolution
        Parameter: 
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
            ch_eis: 3 x n_freq Matrix: [freq, Real, Imag]
        Returen:
            tau_vec: time domain vector
            x: DRT result
            n_extend: number of extend RLC parameters
        ==================================================
    '''
    ## Freq domain data prepare
    
    n_freq = len(f)
    # freq_vec = f
    # Z_exp = Z
    freq_vec = np.flip(f)
    Z_exp = np.flip(Z)

    '''Hyper Parameters'''
    # Time domain parameters
    log_tau_min = np.log10(1/(2*np.pi*freq_vec[0]))  
    log_tau_max = np.log10(1/(2*np.pi*freq_vec[-1])) 
    # log_tau_min = np.log10(1/(freq_vec[0]))  
    # log_tau_max = np.log10(1/(freq_vec[-1]))   
    n_tau = n_freq

    # tau_vec = 1/(2*np.pi*freq_vec)
    # tau_vec = 1/(2*np.pi*100*freq_vec)
    # tau_vec = np.logspace(-4,4, n_tau, endpoint=True)
    tau_vec = np.logspace(log_tau_min, log_tau_max, num = n_tau, endpoint=True)

    # log_tau = np.log(tau_vec)
    # tau_vec = np.logspace(-6, 0, n_tau, endpoint=True)
    # freq_vec = np.flip(np.logspace(0, 6, n_freq, endpoint=True))
    


    # Discretization matrices Parameters
    # Use RBF Kernel to initialize the A matrix
    RBF_shape_control = 'FWHM Coefficient' 
    RBF_coeff = 0.5
    # RBF_type = 'Piecewise Linear'
    RBF_type = 'Gaussian'
    # RBF_type = 'C0 Matern'
    # RBF_type = 'C2 Matern'
    # RBF_type = 'C4 Matern'
    # RBF_type = 'C6 Matern'
    # RBF_type = 'Inverse Quadratic'

    # Cross-validation Method for optimize lambda (Tikhonov regularization parameter) 
    cv_type = 'GCV'     # Generalized Cross Validation
    # cv_type = 'mGCV'    # Modified Generalized Cross Validation
    # cv_type = 'rGCV'    # Robust Generalized Cross Validation
    # cv_type = 'LC'      # L-curve
    # cv_type = 're-im'   # Real-Imaginary discrepancy
    # cv_type = 'kf'      # k-fold cross-validation

    '''Compute the RBF Shape Parameter Epsilon'''
    epsilon = drt.basics.compute_epsilon(freq_vec, RBF_coeff, RBF_type, RBF_shape_control)

    # logger.info(f"{epsilon}")
    # return
    '''Compute the discretization matrices'''
    A_re = drt.basics.assemble_A_re(freq_vec, tau_vec, epsilon, RBF_type)
    n_extend = np.sum(RLC_Flag)   
    if RLC_Flag[2]:
        A_re_C_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_C_0, A_re)) 
    if RLC_Flag[1]:
        A_re_L_0    = np.zeros((n_freq, 1)) 
        A_re        = np.hstack((A_re_L_0, A_re))
    if RLC_Flag[0]:
        A_re_R_inf  = np.ones((n_freq, 1))
        A_re        = np.hstack((A_re_R_inf, A_re)) 

    A_im = drt.basics.assemble_A_im(freq_vec, tau_vec, epsilon, RBF_type)
    if RLC_Flag[2]:
        A_im_C_0    = -1/(2*np.pi*freq_vec.reshape(-1,1))
        A_im        = np.hstack((A_im_C_0, A_im))
    if RLC_Flag[1]:
        A_im_L_0    = 2*np.pi*freq_vec.reshape(-1,1)
        A_im        = np.hstack((A_im_L_0, A_im))
    if RLC_Flag[0]:
        A_im_R_inf  = np.zeros((n_freq, 1)) 
        A_im        = np.hstack((A_im_R_inf, A_im))

    A = np.vstack((A_re, A_im))
     


    '''Compute the differentiation matrices for Tiknonov regularization'''
    M2 = np.zeros((n_tau+n_extend, n_tau+n_extend))
    M2[n_extend:,n_extend:] = drt.basics.assemble_M_2(tau_vec, epsilon, RBF_type)

    '''Optimize lambda'''
    if custom_lambda is None:
        log_lambda_init = -3 # ln(lambda_init = 0.001)
        # lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", RLC_Flag[1], log_lambda_init, cv_type)
        lambda_opt = drt.basics.optimal_lambda(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, "Combined Re-Im Data", 0, log_lambda_init, cv_type)
    else: 
        lambda_opt = custom_lambda
    # logger.info(f"Lambda: {lambda_opt}")
    '''Deconvolve The DRT from the EIS Data'''
    # Set Bound Constraints
    # lb = np.zeros([n_tau+n_extend])
    # bound_mat = np.eye(lb.shape[0])
    H_combined, c_combined = drt.basics.quad_format_combined(A_re, A_im, np.real(Z_exp), np.imag(Z_exp), M2, lambda_opt)
    G = matrix(-np.identity(Z_exp.shape[0]+n_extend))
    h = matrix(np.zeros(Z_exp.shape[0]+n_extend))


    # Deconvolved DRT - Old
    sol = solvers.qp(matrix(H_combined), matrix(c_combined), G, h)
    x = np.array(sol['x'])

    # Deconvolved DRT - New
    # x_var = cp.Variable(H_combined.shape[0])
    # objective = cp.Minimize(0.5 * cp.quad_form(x_var, H_combined) + c_combined.T @ x_var)
    # constraints = [x_var >= 0]
    # prob = cp.Problem(objective, constraints)
    # prob.solve(solver=cp.ECOS, verbose=False,
    #        abstol=1e-6, reltol=1e-6)

    # x = x_var.value

    # Output layer
    H = A@x
    H = H[:n_freq] + 1j*H[n_freq:]
    H = np.flip(H).flatten()
    R_i = x[n_extend:].flatten() / 2
    C_i = tau_vec/R_i

    return R_i, C_i, tau_vec, H, x[:n_extend].flatten(), lambda_opt



def Tik_Analysis_Batch(chData, RLC_Flag=[False, False, False], custom_lambda = None):
    '''==================================================
        DRT Analysis for Batch Data
        Parameter: 
            chData:     list of tuples (f, Z) for each channel
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            results:    list of tuples (R_i, C_i, tau_i, H, res_ReZ, res_ImZ) for each channel
        ==================================================
    '''
    DRTdata = []
    H_list = []
    f = chData[0,0,:]
    for i in range(chData.shape[0]):
        _Z = chData[i,1,:] + 1j*chData[i,2,:]
        R_i, C_i, tau_i, H, _, _= TikDRT(f, _Z, RLC_Flag, custom_lambda)
        DRTdata.append((np.array([R_i, C_i, tau_i])))
        H_list.append(H)
    return DRTdata, H_list

def Tik_Res(Z, H):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return res_ReZ, res_ImZ



## Plot

In [4]:

def DRT_Plot_Batch(DRTdata):

    
    fig = plt.figure()
    axis1 = fig.add_subplot(121)
    axis2 = fig.add_subplot(122)


    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(DRTdata)):
        ch_drt = DRTdata[i]
        _color = cmap(i/len(DRTdata))

        axis1.plot(ch_drt[2,:], ch_drt[0,:], color=_color, linewidth=2, alpha=0.5)
        axis2.plot(ch_drt[2,:], 1/ch_drt[1,:], color=_color, linewidth=2, alpha=0.5)

    axis1.set_xscale('log')
    axis1.set_yscale('log')
    axis2.set_xscale('log')
    axis2.set_yscale('log')


# Simulation

## Monte-Carlo

### Data Simulation

In [63]:
# ECM_TYPE = 'R-(R||C)-(R||C)'
# ECM_TYPE = 'Randle'
ECM_TYPE = 'R-((R-Q)||Q)'
# ECM_TYPE = 'R-((R-Q)||Q)-5000'


if ECM_TYPE == 'R-(R||C)-(R||C)':
    # Element
    R1 = 200; # Ω
    R2 = 100; # Ω
    R0 = 70;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(-2,2,41);  # Hz

    # Calculation of the impedance dataset 
    Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'Randle':
    R0 = 70

    R1 = 10000 
    C1 = 2.5e-9

    Y1 = 1e-5 
    
    n1 = 0.66
    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Z_sim = np.array([ R0 + (R1+Q1(w))/(1+1j*w*C1*(R1+Q1(w))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag
        
elif ECM_TYPE == 'R-((R-Q)||Q)':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'R-((R-Q)||Q)-5000':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



plt.figure()
plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')


### Run

In [75]:
if f.shape[0] < 1000:
    # R_i, C_i, tau_i, H, res_ReZ, res_ImZ = Loe_Analysis_Single(f, Z_sim, REALFLAG=True)
    # svd_L = Loe_singularity_analysis(f, Z_sim, REALFLAG=True)

    # plt.figure()
    # plt.loglog(tau_i,R_i, '.')


    _n_noise = 100
    _drt_noise = []

    Z_org = Z_sim
    # Z_org = Z_sim_noise

    fig = plt.figure()
    axis = fig.add_subplot(111)



    for i in range(_n_noise):
        _Z_noise = Z_org + np.random.normal(0, 0.0001, Z_org.shape) * Z_org.real + np.random.normal(0, 0.0001, Z_org.shape) * Z_org.imag
        
        _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(f, _Z_noise, REALFLAG=True)
        _drt_noise.append(np.array([_R_i, _C_i, _tau_i]))
        axis.scatter(_tau_i[1:-1],_R_i[1:-1], s=1, color='blue')
        axis.scatter(_tau_i[1:-1],_R_i[1:-1]*_R_i.shape[0], s=1,color='red')
        # plt.loglog(_tau_i,_R_i, '.',color='gray')
        # plt.loglog(_tau_i,_R_i*_R_i.shape[0], '.',color='gray')


        
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(f, Z_org, REALFLAG=True)
    _drt_noise.append(np.array([_R_i, _C_i, _tau_i]))
    # axis.scatter(_tau_i,_R_i, color='red')
    # axis.scatter(_tau_i,_R_i*R_i.shape[0], color='red')

    axis.set_xscale('log')
    axis.set_yscale('log')

## Bootstrap

### Data Simulation

In [338]:
# ECM_TYPE = 'R-(R||C)-(R||C)'
# ECM_TYPE = 'Randle'
# ECM_TYPE = 'R-((R-Q2)||Q1)-5000'
ECM_TYPE = 'R-(Q1 || (R-W))-5000'
# ECM_TYPE = '((R-(C1||W)) || C2)-5000'


if ECM_TYPE == 'R-(R||C)-(R||C)':
    # Element
    R1 = 200; # Ω
    R2 = 100; # Ω
    R0 = 70;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(-2,2,41);  # Hz

    # Calculation of the impedance dataset 
    Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



elif ECM_TYPE == 'R-((R-Q2)||Q1)-5000':
    R0 = 1000

    R1 = 10000
    
    Y1 = 1e-9
    n1 = 1
    
    Y2 = 1e-5
    n2 = 0.6

    f = np.logspace(0,6,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag


elif ECM_TYPE == '((R-(C1||W)) || C2)-5000':
    R0 = 1000

    C1 = 1e-8
    C2 = 1e-10

    
    WR = 1e5
    WT = 1e-3
    n2 = 0.6

    f = np.logspace(0,8,5000);  # Hz 

    W1 = lambda x: WR/((WT*1j*x)**n2)
    Q1 = lambda x: 1/(C1*(1j*x))
    Q2 = lambda x: 1/(C2*(1j*x))

    Z_sim = np.array([ 1/( 1/Q2(w) + 1/( R0+ 1/(1/Q1(w)+1/W1(w)) ) ) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag





elif ECM_TYPE == 'R-(Q1 || (R-W))-5000':
    R0 = 1000

    R1 = 1e5
    
    Y1 = 1e-10
    n1 = 0.9
    
    WR = 100
    WT = 1e-7
    n2 = 0.6

    f = np.logspace(1,6,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: WR/((1j*x*WT)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag




plt.figure()
plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')
plt.title(ECM_TYPE)


Text(0.5, 1.0, 'R-(Q1 || (R-W))-5000')

### Run

In [339]:
def stratified_subsample(total_size, n_subsample, idx_base=1000, seed=None):
    """
    将 total_size 个点均匀分成 n_subsample 段，每段随机取 1 个索引，无放回采样。
    """
    if seed is not None:
        np.random.seed(seed)
    
    indices = np.arange(idx_base, total_size)
    bins = np.array_split(indices, n_subsample)
    sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
    
    return np.array(sampled_indices)

In [340]:

# Z_org = Z_sim
Z_org = Z_sim_noise

_f = f
_Z = Z_org



n_batch = 100
n_point = 100

_f_bootstrap = []
_Z_bootstrap = []
for i in range(n_batch):
    _idx = stratified_subsample(_f.shape[0], n_point)
    _f_bootstrap.append(_f[_idx])
    _Z_bootstrap.append(_Z[_idx])

_f = np.stack(_f_bootstrap, axis=0)
_Z = np.stack(_Z_bootstrap, axis=0)

if False:
    plt.figure()
    for i in range(_f.shape[0]):
        plt.loglog(_f[i,:], np.abs(_Z[i,:]))


In [ ]:
_drt_list = []

fig = plt.figure()
axis = fig.add_subplot(111)
for i in range(_f.shape[0]):
    # if i <20: continue
    _R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(_f[i,:], _Z[i,:], REALFLAG=True)
    _drt_list.append(np.array([_R_i, _C_i, _tau_i]))

    # R_0 = _R_i[0]
    # C_0 = _C_i[-1]
    # Z_cal = _Z[i,:] - R_0 - 1/(2j*np.pi*_f[i,:]*C_0)
    # _R_cal, _C_cal, _tau_cal, _, _, _ = Loe_Analysis_Single(_f[i,:], Z_cal, REALFLAG=True)



    # axis.scatter(_tau_i,_R_i, s=2, color = 'gray')
    # axis.scatter(_R_i, _C_i, s=2, color = 'gray')
    # axis.scatter(_tau_i[0],_R_i[0], s=2)
    # axis.scatter(_tau_i[0],_R_i[0]*_R_i.shape[0], s=2)

    ## Last Point Discuss
    # axis.scatter(_tau_i[1:-4],_R_i[1:-4], s=2, color = 'gray')
    # axis.scatter(_tau_i[0],_R_i[0], s=2, color = 'red')
    # axis.scatter(_tau_i[-4],_R_i[-4], s=2, color='blue')
    # axis.scatter(_tau_i[-3],_R_i[-3], s=2, color='green')
    # axis.scatter(_tau_i[-2],_R_i[-2], s=2, color='orange')
    # axis.scatter(_tau_i[-1],_R_i[-1], s=2, color='red')

    ## Order Correction Discuss - Conclusion: correction should NOT be applied here
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10], s=2, color='red')
    # axis.scatter(_tau_i[1:-10],_R_i[1:-10]*_R_i.shape[0], s=2, color='blue')

    # ## RC Discussion
    # axis.scatter(_R_i[1:-1], _C_i[1:-1], s=2, color = 'gray')
    # axis.scatter(_R_i[-1], _C_i[-1], s=2, color = 'red')
    # axis.scatter(_R_i[0], _C_i[0], s=2, color = 'orange')


    ## R/C Discussion
    # axis.scatter(_R_i[1:-1]*_C_i[1:-1], _R_i[1:-1]/_C_i[1:-1], s=2, color = 'gray')
    # axis.scatter(_R_i[-1]*_C_i[-1], _R_i[-1]/_C_i[-1], s=2, color = 'red')
    # axis.scatter(_R_i[0]*_C_i[0], _R_i[0]/_C_i[0], s=2, color = 'orange')
    
    # axis.scatter(_R_cal[1:-4], _C_cal[1:-4], s=2, color = 'gray')
    # axis.scatter(_R_cal[-1], _C_cal[-1], s=2, color = 'red')
    # axis.scatter(_R_cal[0], _C_cal[0], s=2, color = 'orange')

    ## R/C/(x+1/x)
    _tt = _R_i[:]*_C_i[:]
    _yy = _R_i[:] / _C_i[:]
    # _yy = _yy / (((_tt/1e0)**-1) + ((_tt/1e0)**1))
    # _yy = _yy / (((_tt/1e-4)**-1) + ((_tt/1e-4)**1))
    # _yy = _yy / (((_tt/1e-3)**-1) + ((_tt/1e-7)**1))
    axis.scatter(_tt[1:-1], _yy[1:-1], s=2, color = 'gray')
    axis.scatter(_tt[-1], _yy[-1], s=2, color = 'red')
    axis.scatter(_tt[0], _yy[0], s=2, color = 'orange')
    
    axis.scatter(_tt[0], _yy[0], s=2, color = 'orange')
    


# axis.set_xlim([1e-10,1e2])
# axis.set_ylim([1e0,1e10])
axis.set_xscale('log')
axis.set_yscale('log')
axis.set_aspect('equal')

In [342]:
print(_C_i[-1])
print(_R_i[0])

9.144686990918363e-08
1775.1350230542503


## RC Bias

### Data simulation

In [342]:
# ECM_TYPE = 'R-(R||C)-(R||C)'
# ECM_TYPE = 'Randle'
ECM_TYPE = 'R-((R-Q)||Q)'
# ECM_TYPE = 'R-((R-Q)||Q)-5000'


if ECM_TYPE == 'R-(R||C)-(R||C)':
    # Element
    R1 = 200; # Ω
    R2 = 100; # Ω
    R0 = 70;  # Ω

    C1 = 2.5e-3; # 
    C2 = 1e-4;   # 

    # tau1 = R1*C1;  # s
    # tau2 = R2*C2;  # s

    f = np.logspace(-2,2,41);  # Hz

    # Calculation of the impedance dataset 
    Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'Randle':
    R0 = 70

    R1 = 10000 
    C1 = 2.5e-9

    Y1 = 1e-5 
    
    n1 = 0.66
    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Z_sim = np.array([ R0 + (R1+Q1(w))/(1+1j*w*C1*(R1+Q1(w))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag
        
elif ECM_TYPE == 'R-((R-Q)||Q)':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,61);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag

elif ECM_TYPE == 'R-((R-Q)||Q)-5000':
    R0 = 70

    R1 = 50000 
    
    Y1 = 1e-8
    n1 = 0.8
    
    Y2 = 1e-5 
    n2 = 0.66

    f = np.logspace(-1,5,5000);  # Hz 

    Q1 = lambda x: 1/(Y1*(1j*x)**n1)
    Q2 = lambda x: 1/(Y2*(1j*x)**n2)

    Z_sim = np.array([ R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
    # Z_sim_noise = Z_sim + np.random.normal(0, 0.01, Z_sim.shape) * Z_sim
    Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



plt.figure()
plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')


### Run

In [343]:

Z_org = Z_sim

_R_i, _C_i, _tau_i, _, _, _ = Loe_Analysis_Single(f, Z_org, REALFLAG=True)

print(_C_i[-1])
print(_R_i[0])


7.647711752979606e-05
180.17607830131084


In [344]:
R_0 = _R_i[0]
C_0 = _C_i[-1]
Z_cal = Z_org - R_0 - 1/(2j*np.pi*f*C_0)
_R_cal, _C_cal, _tau_cal, _, _, _ = Loe_Analysis_Single(f, Z_cal, REALFLAG=True)


In [345]:
fig = plt.figure()
axis = fig.add_subplot(111)
axis.scatter(_R_i, _C_i, s=10, color = 'red')
# axis.scatter(_tau_cal,_R_cal, s=10, color = 'blue')

axis.set_xscale('log')
axis.set_yscale('log')


# Draft

In [348]:
import numpy as np
import matplotlib.pyplot as plt
from lmfit.models import GaussianModel

# 合成示例数据
x = np.linspace(0, 20, 500)
np.random.seed(0)
y = (1.5 * np.exp(-(x - 6)**2 / (2 * 1.2**2)) +
     2.0 * np.exp(-(x - 13)**2 / (2 * 1.5**2)) +
     0.05 * np.random.randn(len(x)))

# 模型定义（两个高斯）
model = GaussianModel(prefix='g1_') + GaussianModel(prefix='g2_')

# 参数初始值估计
params = model.make_params()
params['g1_center'].set(6)
params['g1_sigma'].set(1)
params['g1_amplitude'].set(1)

params['g2_center'].set(13)
params['g2_sigma'].set(1)
params['g2_amplitude'].set(2)

# 拟合
result = model.fit(y, params, x=x)

# 拟合结果和每个成分画图
components = result.eval_components(x=x)

plt.plot(x, y, 'b', label='data')
plt.plot(x, result.best_fit, 'r-', label='fit')
plt.plot(x, components['g1_'], 'g--', label='g1')
plt.plot(x, components['g2_'], 'm--', label='g2')
plt.legend()
plt.show()


c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
